<a href="https://colab.research.google.com/github/jinwooahnKHU/NLP/blob/main/sentence_pair_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 문서 쌍 분류 모델 + NLI

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 57 kB 7.0 MB/s 
     |████████████████████████████████| 806 kB 21.5 MB/s 
     |████████████████████████████████| 2.8 MB 48.5 MB/s 
     |████████████████████████████████| 8.0 MB 31.1 MB/s 
     |████████████████████████████████| 398 kB 65.2 MB/s 
     |████████████████████████████████| 136 kB 70.3 MB/s 
     |████████████████████████████████| 829 kB 52.9 MB/s 
     |████████████████████████████████| 636 kB 49.4 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:38tcmalloc: large alloc 1147494400 bytes == 0x39c84000 @  0x7f9ebe0fc615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 19 kB/s 
     |████████

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments

args= ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_task_name="pair-classification",
    downstream_corpus_name="klue-nli",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-paircls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=5,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [4]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


set seed: 7


In [5]:
#말뭉치 내려받기기(KLUE-NLI from UPSTAGE)
# download klue-nli(corpus_name) at downstream_corpus_root_dir (/root/Korpora)
nlpbook.download_downstream_dataset(args)

Downloading: 100%|██████████| 12.3M/12.3M [00:00<00:00, 49.5MB/s]
Downloading: 100%|██████████| 1.47M/1.47M [00:00<00:00, 51.3MB/s]


In [6]:
# prepare tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [7]:
# train dataset
from ratsnlp.nlpbook.paircls import KlueNLICorpus
from ratsnlp.nlpbook.classification import ClassificationDataset

# make corpus into trainable classification features (sentences to input_id, attention mask, token_type_ids, label)
# entailment : 0, contradiction : 1, neutral : 2

corpus = KlueNLICorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/klue-nli
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/klue-nli/klue_nli_train.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 9.809 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence A, B: 100분간 잘껄 그래도 소닉붐땜에 2점준다 + 100분간 잤다.
INFO:ratsnlp:tokens: [CLS] 100 ##분간 잘 ##껄 그래도 소 ##닉 ##붐 ##땜에 2 ##점 ##준다 [SEP] 100 ##분간 잤 ##다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: contradiction
INFO:ratsnlp:features: ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=1)

In [9]:
# train data loader 
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn= nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [10]:
#test dataloader
val_dataset = ClassificationDataset(
    args= args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)

val_dataloader=DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler = SequentialSampler(val_dataset),
    collate_fn = nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/klue-nli
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/klue-nli/klue_nli_dev.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 1.154 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence A, B: 10명이 함께 사용하기 불편함없이 만족했다. + 10명이 함께 사용하기 불편함이 많았다.
INFO:ratsnlp:tokens: [CLS] 10명 ##이 함께 사용 ##하기 불편 ##함 ##없이 만족 ##했다 . [SEP] 10명 ##이 함께 사용 ##하기 불편 ##함이 많았 ##다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: contradiction
INFO:ratsnlp:features: ClassificationFeatures(input_ids=[2, 21000, 4017, 9158, 9021, 8268, 10588, 4421, 8281, 14184, 8258, 17, 3, 21000, 4017, 9158, 9021, 8268, 10588, 11467, 14338, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
# initialize model
from transformers import BertConfig, BertForSequenceClassification

pretrained_model_config=BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

In [12]:
#BertForSequenceClassification : pretrained kcbert에 문서 분류용 태스크 모듈을 덧붙힘
model=BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [13]:
#define task
from ratsnlp.nlpbook.classification import ClassificationTask
task= ClassificationTask(model, args)

In [14]:
trainer=nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [15]:
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.683   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Inference

In [16]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

args=ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-paircls",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-paircls/epoch=0-val_loss=0.83.ckpt


In [17]:
import torch
fine_tuned_model_ckpt=torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)

In [18]:
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

In [19]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification(pretrained_model_config)

In [20]:
# put ckpt 
model.load_state_dict({k.replace("model.",""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [21]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
def inference_fn(premise, hypothesis):
  inputs= tokenizer(
      [(premise, hypothesis)],
      max_lengths=args.max_seq_length,
      padding="max_length",
      truncation=True,
  )

  with torch.no_grads():
    outputs=model(**{k:torch.tensor(v) for k, v in inputs.items()}) #change input to tensor
    prob = outputs.logits.softmax(dim=1) #softmax value to logits
    entailment_prob=round(prob[0][0].items(),2)
    contradiction_prob=round(prob[0][1].items(), 2)
    neutral_prob=round(prob[0][2].items(),2)

    if torch.argmax(prob)==0:
      pred= "참 (entailment)"
    elif torch.argmax(prob) == 1:
      pred = "거짓 (contradiction)"
    else:
      pred="중립 (neutral)"
    
  return {
      'premise' : premise,
      'hypothesis' : hypothesis,
      'prediction' : pred,
      'entailment_data' : f"참 {entailment_prob}",
      'contradiction_data' : f"거짓 {contradiction_prob}",
      'neutral_data' : f"중립 {neutral_prob}",
      'entailment_width' : f"{entailment_prob * 100}%",
      'contradiction_width' : f"{contradiction_prob * 100}%",
      'neutral_width' : f"{neutral_prob * 100}%",
  }

In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app

app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://748b-34-83-222-25.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [11/May/2022 03:57:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/May/2022 03:57:35] "GET /favicon.ico HTTP/1.1" 404 -
